In [1]:
from batchglm.unit_test.test_jacobians_glm_all_tf2 import *

/home/mario/miniconda3/envs/batchglm_tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mario/miniconda3/envs/batchglm_tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mario/miniconda3/envs/batchglm_tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mario/miniconda3/envs/

In [2]:
import tensorflow as tf
def _ll(eta_loc, eta_scale, loc, scale, x, n_features):

        # Log-likelihood:
        log_r_plus_mu = tf.math.log(scale + loc)
        if isinstance(x, tf.SparseTensor):
            log_probs_sparse = x.__mul__(eta_loc - log_r_plus_mu)
            log_probs_dense = tf.math.lgamma(tf.sparse.add(x, scale)) - \
                              tf.math.lgamma(tf.sparse.add(x, tf.ones(shape=x.dense_shape, dtype=self.ll_dtype))) - \
                              tf.math.lgamma(scale) + \
                              tf.multiply(scale, eta_scale - log_r_plus_mu)
            log_probs = tf.sparse.add(log_probs_sparse, log_probs_dense)
            # log_probs.set_shape([None, n_features])  # need as shape completely lost.
        else:
            # print(scale.get_shape())
            # print(x.get_shape())
            log_probs = tf.math.lgamma(scale + x) - \
                        tf.math.lgamma(x + tf.ones_like(x)) - \
                        tf.math.lgamma(scale) + \
                        tf.multiply(x, eta_loc - log_r_plus_mu) + \
                        tf.multiply(scale, eta_scale - log_r_plus_mu)

        # log_probs = self.tf_clip_param(log_probs, "log_probs")
        return log_probs
def analytic():
    x_minus_loc = tf.subtract(x,loc)
    scalar_one = tf.constant(1, shape=(), dtype=tf.dtypes.float64)
    a_model = tf.divide(tf.multiply(scale, x_minus_loc), tf.add(loc, scale))
    print("a_model: ", a_model)
    r_plus_mu = scale + loc
    scale_plus_x = scale + x
    # Define graphs for individual terms of constant term of hessian:
    const1 = tf.subtract(
        tf.math.digamma(x=scale_plus_x),
        tf.math.digamma(x=scale)
    )
    const2 = tf.negative(scale_plus_x / r_plus_mu)
    const3 = tf.add(
        tf.math.log(scale),
        scalar_one - tf.math.log(r_plus_mu)
    )
    #const = const1 + const2
    #const +=const3
    const = tf.add_n([const1, const2, const3])  # [observations, features]
    b_model = scale * const
    a_model = tf.matmul(tf.transpose(a_model), design_loc)
    b_model = tf.matmul(tf.transpose(b_model), design_scale)
    print(x.dtype)
    print(loc.dtype)
    print(scale.dtype)
    print(x_minus_loc.dtype)
    return tf.concat([a_model, b_model], 1)


In [3]:
unittest.main(argv=['first-arg-is-ignored'], exit = False)

E0915 12:43:39.468703 140048598378304 test_jacobians_glm_all_tf2.py:156] Test_Jacobians_GLM_NB.test_compute_jacobians_nb()
E

1
ja ich habe den mist initialisiert
float64



ERROR: test_compute_jacobians_nb (batchglm.unit_test.test_jacobians_glm_all_tf2.Test_Jacobians_GLM_NB)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/mario/PSC19/batchglm_tf2/batchglm/unit_test/test_jacobians_glm_all_tf2.py", line 159, in test_compute_jacobians_nb
    self._test_compute_jacobians(sparse=False)
  File "/home/mario/PSC19/batchglm_tf2/batchglm/unit_test/test_jacobians_glm_all_tf2.py", line 148, in _test_compute_jacobians
    self.compare_jacs(design="~ 1 + condition + batch", sparse=sparse)
  File "/home/mario/PSC19/batchglm_tf2/batchglm/unit_test/test_jacobians_glm_all_tf2.py", line 118, in compare_jacs
    J_analytic = self.get_jacs(input_data)
  File "/home/mario/PSC19/batchglm_tf2/batchglm/unit_test/test_jacobians_glm_all_tf2.py", line 74, in get_jacs
    "autograd": pkg_constants.JACOBIAN_MODE == "tf"
  File "/home/mario/PSC19/batchglm_tf2/batchglm/train/tf2/base_glm/estimator.py", line 63, in 

In [4]:
x = np.array([[287,300],[1331,1200]], dtype="float64")
print(type(x))
np.ndarray(shape=(2,2),buffer=x, dtype="float64")

<class 'numpy.ndarray'>


array([[ 287.,  300.],
       [1331., 1200.]])

In [5]:
a_var = tf.constant([[7.15805998, 7.34018684],[0,0]], dtype=tf.dtypes.float64)
b_var = tf.constant([[2.17102654, 0.43241663],[0,0]], dtype=tf.dtypes.float64)
params = tf.concat([a_var,b_var], axis=0)
params = tf.Variable(params)
design_loc = tf.constant([[1,1],[1,0]], dtype=tf.dtypes.float64) #n_obs x coeff
design_scale = tf.constant([[1,1],[1,0]], dtype=tf.dtypes.float64)
x = tf.constant([[931.,299.],[1893.,2783.]], dtype=tf.dtypes.float64) # n_obs x features

with tf.GradientTape() as g:
    g.watch(params)
    a_var = params[0:2]
    b_var = params[2:4]
    print(design_loc)
    print(params)
    eta_loc = tf.matmul(design_loc, a_var)
    eta_scale = tf.matmul(design_scale, b_var)
    loc = tf.exp(eta_loc)
    scale= tf.exp(eta_scale)
    result = _ll(eta_loc,eta_scale,loc,scale, x, 3)
xxx = g.gradient(result, params)


print(tf.negative(xxx))

tf.Tensor(
[[1. 1.]
 [1. 0.]], shape=(2, 2), dtype=float64)
<tf.Variable 'Variable:0' shape=(4, 2) dtype=float64, numpy=
array([[7.15805998, 7.34018684],
       [0.        , 0.        ],
       [2.17102654, 0.43241663],
       [0.        , 0.        ]])>
tf.Tensor(
[[-1.72992901e+00  1.44090677e-08]
 [ 2.39602649e+00  1.24074074e+00]
 [ 1.35365258e-01  5.08817847e-01]
 [-1.02071679e-01  7.30115644e-01]], shape=(4, 2), dtype=float64)


In [6]:
tf.negative(tf.transpose(analytic()))

a_model:  tf.Tensor(
[[-2.39602649 -1.24074074]
 [ 4.12595551  1.24074073]], shape=(2, 2), dtype=float64)
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>


<tf.Tensor: id=184, shape=(4, 2), dtype=float64, numpy=
array([[-1.72992901e+00,  1.44087806e-08],
       [ 2.39602649e+00,  1.24074074e+00],
       [ 1.35365258e-01,  5.08817847e-01],
       [-1.02071679e-01,  7.30115644e-01]])>

In [7]:
a_model:  tf.Tensor(
[[-2.96656217 -1.24074074]
 [ 2.96656217  1.24074074]], shape=(2, 2), dtype=float64)

SyntaxError: invalid syntax (<ipython-input-7-8c801b035616>, line 3)